In [1]:
import torch
import torch.nn as nn
from ultralytics import YOLO

In [2]:
import sys
sys.path.insert(1, '../')
from Models.yolov8cls_path import *
# %run block.ipynb

variants taken from https://github.com/ultralytics/ultralytics/blob/07a5ff9ddca487581035b61ff7678c0f7e0f40d9/ultralytics/cfg/models/v8/yolov8.yaml

### Path:

In [3]:
model1 = Model(num_classes=10)

In [4]:
inp = torch.rand(1,3,640,640)

In [5]:
out = model1(inp)

In [6]:
out.shape

torch.Size([1, 10])

In [7]:
total_params = sum(p.numel() for p in model1.parameters())
total_params

2509712

In [8]:
model2 = Model(num_classes=10, residual_connection=True, CSP=True)

In [9]:
out = model2(inp)

In [10]:
out.shape

torch.Size([1, 10])

In [11]:
total_params = sum(p.numel() for p in model2.parameters())
total_params

1056848

In [12]:
model3 = Model(num_classes=10, residual_connection=True, CSP=True, add_hidden=True, classifyV8=True)

In [13]:
out = model3(inp)

In [14]:
total_params = sum(p.numel() for p in model3.parameters())
total_params

1451098

## Do the parameters match?

### YOLOv2n-cls

In [15]:
model1 = Model(residual_connection=True, CSP=True, add_hidden=True, bottleneck=1.0,
                 num_classes=1000, variant='n', classifyV8=True, dtype=torch.float32)

In [16]:
reference = torch.load('yolov8n-cls.pt')['model'].model

In [17]:
all_parameters_match = True
for i, (p1, p2) in enumerate(zip(model1.parameters(), reference.parameters())):
    all_parameters_math = all_parameters_match and p1.shape == p2.shape

In [18]:
all_parameters_match

True

In [19]:
total_params = sum(p.numel() for p in model1.parameters())
total_params

2719288

In [20]:
total_params_reference = sum(p.numel() for p in reference.parameters())
total_params_reference

2719288

In [21]:
total_params == total_params_reference

True

### YOLOv2s-cls

In [22]:
reference = torch.load('yolov8s-cls.pt')['model'].model

In [23]:
model1 = Model(residual_connection=True, CSP=True, add_hidden=True, bottleneck=1.0,
                 num_classes=1000, variant='s', classifyV8=True, dtype=torch.float32)

In [24]:
all_parameters_match = True
for i, (p1, p2) in enumerate(zip(model1.parameters(), reference.parameters())):
    all_parameters_math = all_parameters_match and p1.shape == p2.shape

In [25]:
all_parameters_math

True

In [26]:
total_params = sum(p.numel() for p in model1.parameters())
total_params

6361736

In [27]:
total_params_reference = sum(p.numel() for p in reference.parameters())
total_params_reference

6361736

In [28]:
total_params == total_params_reference

True